In [8]:
from pandas import read_csv, DataFrame
from factor_analyzer import FactorAnalyzer

project_home = '/Users/catcamacho/Box/SNAP/BABIES/misc/'
ibq_data = read_csv(project_home + '2kmeans_group_fulldata.csv', index_col=0)
features=ibq_data.iloc[:,20:34]

fa = FactorAnalyzer()
fa.analyze(features,6,rotation='varimax', normalize=False)
print(fa.get_factor_variance())

                 Factor1   Factor2   Factor3   Factor4   Factor5   Factor6
SS Loadings     1.854892  1.475507  1.141746  1.238987  0.931417  0.829951
Proportion Var  0.132492  0.105393  0.081553  0.088499  0.066530  0.059282
Cumulative Var  0.132492  0.237886  0.319439  0.407938  0.474468  0.533750


In [9]:
loadings = fa.loadings
loadings = loadings.sort_values('Factor1')
print(loadings)

                              Factor1   Factor2   Factor3   Factor4   Factor5  \
sadness_std                 -0.047760  0.594649  0.130150  0.065057 -0.047599   
distress_to_limitations_std  0.007590  0.817534 -0.185897  0.208984  0.047260   
falling_reactivity_std       0.016515  0.367694 -0.323240  0.199732  0.099369   
cuddliness_std               0.027433 -0.122558  0.123970 -0.272517  0.076129   
activity_level_std           0.056225  0.130694 -0.005998  0.984372  0.051973   
low_intensity_pleasure_std   0.096481 -0.125312  0.808265 -0.023987  0.097699   
duration_of_orienting_std    0.105270 -0.049027  0.225321 -0.053344  0.213873   
soothability_std             0.138217 -0.176757  0.018537 -0.212371  0.174334   
fear_std                     0.181203  0.431555  0.205983  0.097425 -0.217540   
perceptual_sensitivity_std   0.193375  0.101811  0.432111  0.120759  0.326936   
approach_std                 0.262377  0.013374  0.112501  0.089326  0.791518   
vocal_reactivity_std        